In [49]:
from scipy.io import wavfile
import numpy as np
from scipy.fftpack import fft

In [50]:
# загружаем файл wav
filename = 'File_121_2023.wav'
filename = 'File_06_p2_2023.wav'
frequency = 700
sampling_rate, data = wavfile.read(filename)


In [51]:
# выводим частоту дискретизации
print(f"Частота дискретизации: {sampling_rate} Гц")

# выводим форму массива
print(f"Форма массива: {data.shape}")

# преобразуем массив в тип float и делим на максимальное значение амплитуды
data = data.astype(np.float32) / np.iinfo(data.dtype).max

# выводим первые 10 элементов массива
print("Первые 10 элементов массива:")
print(data[:10])

Частота дискретизации: 8000 Гц
Форма массива: (851200,)
Первые 10 элементов массива:
[-0.45844904 -0.27961057  0.15982543  0.47257912  0.15775017 -0.3970153
  0.15225685 -0.12839137 -0.30527666  0.37186804]


In [52]:
window_size = int(sampling_rate * 0.1)  # размер окна в отсчетах (100 мс)

freqs_list = []
word = []

delta = []

for i in range(0, len(data), window_size):
    # выделяем отрезок длиной window_size
    segment = data[i:i+window_size]

    # вычисляем одностороннее преобразование Фурье
    spectrum = np.abs(fft(segment)[:window_size//2])

    # находим индекс частоты, соответствующей максимальному значению спектра
    max_index = np.argmax(spectrum)

    # находим частоту, соответствующую этому индексу
    freq = max_index * sampling_rate / window_size

    # добавляем частоту в список
    freqs_list.append(freq)

    # находим среднее значение мощности сегмента
    mean_power = np.mean(np.power(segment, 2))

    # находим значение максимальной спектральной компоненты
    max_spectrum = spectrum[max_index]

    # вычисляем разницу между максимальной величиной спектра и средней величиной мощности сегмента
    db_difference = abs(10 * np.log10(max_spectrum / mean_power))
    delta.append(db_difference)
    if frequency - 1 <= freq <= frequency + 1 and db_difference >= 5:
        word.append(1)
    else:
        word.append(0)



print("Список частот, соответствующих максимуму спектра на отрезках по 100 мс:")
print(freqs_list)


Список частот, соответствующих максимуму спектра на отрезках по 100 мс:
[1860.0, 700.0, 500.0, 130.0, 1640.0, 2430.0, 700.0, 700.0, 3350.0, 700.0, 700.0, 1990.0, 700.0, 700.0, 700.0, 700.0, 1530.0, 700.0, 700.0, 400.0, 1530.0, 700.0, 1390.0, 850.0, 2210.0, 700.0, 700.0, 3050.0, 700.0, 1020.0, 3340.0, 700.0, 3320.0, 700.0, 700.0, 350.0, 700.0, 700.0, 700.0, 1480.0, 2730.0, 700.0, 700.0, 960.0, 2630.0, 700.0, 700.0, 700.0, 3140.0, 1210.0, 700.0, 1860.0, 3170.0, 1250.0, 2580.0, 3860.0, 710.0, 700.0, 700.0, 700.0, 1460.0, 1820.0, 700.0, 700.0, 460.0, 700.0, 700.0, 700.0, 1410.0, 700.0, 1500.0, 520.0, 3810.0, 700.0, 700.0, 2730.0, 2150.0, 2440.0, 130.0, 700.0, 2090.0, 700.0, 700.0, 3270.0, 700.0, 700.0, 700.0, 920.0, 3060.0, 700.0, 700.0, 2110.0, 2100.0, 700.0, 320.0, 3820.0, 1870.0, 700.0, 700.0, 1410.0, 3800.0, 3320.0, 3010.0, 700.0, 1590.0, 700.0, 700.0, 700.0, 1640.0, 1850.0, 700.0, 2560.0, 3050.0, 700.0, 700.0, 1970.0, 40.0, 700.0, 3820.0, 0.0, 3710.0, 700.0, 700.0, 700.0, 2930.0, 3190

In [53]:
delta

[24.304232597351074,
 26.409990787506104,
 24.08303737640381,
 24.10426616668701,
 23.914384841918945,
 24.066710472106934,
 26.254825592041016,
 27.506685256958008,
 24.190514087677002,
 26.540629863739014,
 27.0021915435791,
 23.817906379699707,
 25.978519916534424,
 26.804168224334717,
 26.099607944488525,
 26.753652095794678,
 23.448505401611328,
 26.89929962158203,
 26.44540548324585,
 23.990771770477295,
 24.338181018829346,
 26.784379482269287,
 24.39800500869751,
 24.36147928237915,
 23.472590446472168,
 27.407355308532715,
 26.5490460395813,
 24.21403169631958,
 26.625540256500244,
 23.845736980438232,
 24.7707200050354,
 27.46098518371582,
 23.751940727233887,
 27.041544914245605,
 26.78244113922119,
 23.874423503875732,
 26.07133388519287,
 26.94058656692505,
 26.37157440185547,
 23.929812908172607,
 24.86685276031494,
 25.937800407409668,
 27.12618350982666,
 23.426530361175537,
 23.40587854385376,
 27.042512893676758,
 26.59252643585205,
 26.57973051071167,
 23.81092309951

In [54]:
#padding
remainder = len(word) % 8
if remainder != 0:
    word.extend([0] * (8 - remainder))

print(word)


[0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 

In [55]:
text = ''
for i in range(0, len(word), 8):
    code = int(''.join(map(str, word[i:i+8])), 2)
    print(code)
    str1 = chr(code)
    text += str1
    print(str1)
print(text)
#30

67
C
111
o
100
d
105
i
110
n
103
g
32
 
115
s
116
t
97
a
110
n
100
d
97
a
114
r
100
d
115
s
58
:
32
 
67
C
111
o
100
d
105
i
110
n
103
g
32
 
115
s
116
t
97
a
110
n
100
d
64
@
114
r
100
d
115
s
32
 
97
a
114
r
101
e
32
 
97
a
32
 
115
s
101
e
116
t
32
 
111
o
102
f
32
 
103
g
117
u
105
i
100
d
101
e
108
l
105
i
110
n
101
e
115
s
32
 
97
a
110
n
100
d
32
 
98
b
101
e
115
s
116
t
32
 
112
p
114
r
97
a
99
c
116
t
105
i
99
c
101
e
115
s
32
 
116
t
104
h
97
a
116
t
32
 
97
a
114
r
101
e
32
 
117
u
115
s
101
e
100
d
32
 
116
t
111
o
32
 
99
c
114
r
101
e
97
a
116
t
101
e
32
 
99
c
111
o
110
n
115
s
105
i
115
s
116
t
101
e
110
n
116
t
44
,
32
 
104
h
105
i
103
g
104
h
45
-
113
q
117
u
97
a
108
l
105
i
116
t
121
y
32
 
99
c
111
o
100
d
101
e
46
.
13

Coding standards: Coding stand@rds are a set of guidelines and best practices that are used to create consistent, high-quality code.


In [56]:
err = 30
print(text[err])
print(freqs_list[err * 8 : (err+1) * 8])
print(delta[err * 8 : (err+1) * 8])
print(word[err * 8 : (err+1) * 8])

@
[1120.0, 700.0, 370.0, 3130.0, 540.0, 1950.0, 3040.0, 3170.0]
[23.279812335968018, 26.435158252716064, 24.00700569152832, 23.962669372558594, 24.12971019744873, 23.66985559463501, 23.83169412612915, 25.025229454040527]
[0, 1, 0, 0, 0, 0, 0, 0]
